In [11]:
import os, sys
import rasterio
import argparse
import yaml
import numpy as np
from tqdm import tqdm
from pathlib import Path
import seaborn as sns
import pandas as pd
from argparse import Namespace
os.chdir("/scratch/ewalt/pdm/rs-uncertainty/")

In [12]:
def pjoin(*subs): return Path(os.path.abspath(os.path.join(*subs)))

In [13]:
# load config
args = Namespace()
args.cfg = Path("./config/evaluate_testset.yaml")
with args.cfg.open("r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)

In [52]:
projects = cfg["projects_east"]+cfg["projects_west"]+cfg["projects_north"]
with Path("/scratch/ewalt/pdm/rs-uncertainty/assets/data/pkl/2021-05-18_10-57-45/stats.yaml").open() as f:
    stats = yaml.safe_load(f)
labels_mean = np.array(stats["labels_mean"]).reshape(5,1,1)
labels_std = np.array(stats["labels_std"]).reshape(5,1,1)
# with pjoin(args.cfg["pkl_dir"], "stats.yaml").open("r", encoding="utf-8") as f:
#     stats = yaml.safe_load(f)
# labels_mean = np.array(stats["labels_stats"]["mean"])
# labels_std = np.array(stats["labels_stats"]["std"])

In [70]:
# iterate and standardize
variables = ['P95', 'MeanH', 'Dens', 'Gini', 'Cover']
def add(arr, source, pid):
    count = arr.shape[1]*arr.shape[2]
    arr = arr.reshape(-1,)   
    data["values"].extend(arr.tolist())
    for variable in variables:
        data["variables"].extend([variable for _ in range(int(arr.shape[0]//5))])
    data["sources"].extend([source for _ in range(arr.shape[0])])
    data["states"].extend(["before" for _ in range(arr.shape[0])])
    data["project_id"].extend([pid for _ in range(arr.shape[0])])
# for mean_file in tqdm(cfg["pkl_dir"].glob('*_mean.tif')):
for mean_file in tqdm(Path("/scratch/ewalt/pdm/rs-uncertainty/results/2023-04-05_15-46-48").glob('*_mean.tif')):
    data = {
        "values": [],
        "variables": [],
        "sources": [],
        "states": [],
        "project_id": [],
    }
    # load data
    project = mean_file.stem.split('_')[0]
    if project not in projects: continue
    with rasterio.open(mean_file) as fh:
        mean = fh.read(fh.indexes)
    # with rasterio.open(pjoin(cfg['prediction_dir'], f"{project}_variance.tif")) as fh:
    with rasterio.open(pjoin("/scratch/ewalt/pdm/rs-uncertainty/results/2023-04-05_15-46-48", f"{project}_variance.tif")) as fh:
        variance = fh.read(fh.indexes)
    with rasterio.open(pjoin(cfg['gt_dir'], f"{project}.tif")) as fh:
        gt = fh.read(fh.indexes)
    # standardize
    variance_after = variance/(labels_std)**2
    mean_after = (mean-labels_mean)/labels_std
    gt_after = (gt-labels_mean)/labels_std 
    # add flatten
    add(mean, "before", project)
    add(mean_after, "after", project)
    add(variance, "before", project)
    add(variance_after, "after", project)
    add(gt, "before", project)
    add(gt_after, "after", project)
    data = pd.DataFrame(data)
    data.dropna(inplace=True)
    data.reset_index(drop=True,inplace=True)
    break


0it [00:00, ?it/s]

In [69]:
data

,values,variables,sources,states,project_id
0,NaN,P95,before,before,792
1,NaN,P95,before,before,792
2,NaN,P95,before,before,792
3,NaN,P95,before,before,792
4,NaN,P95,before,before,792
...,...,...,...,...,...
105895675,-3.919639e+06,Cover,after,before,792
105895676,-3.919639e+06,Cover,after,before,792
105895677,-3.919639e+06,Cover,after,before,792
105895678,-3.919639e+06,Cover,after,before,792


In [66]:
17649280*6

105895680

In [ ]:
data_y = {
    "values": 
    "variables":
    "sources": 
    "states":
}

In [34]:
df_y = pd.DataFrame(columns=["values", "source", "set"])

In [35]:
df_y

,values,source,set


AttributeError: 'DataFrame' object has no attribute 'data'

In [37]:
np.random.randn(2,2)

[[1.7408711770185652, -2.3883417454258864],
 [-0.829603509927723, 0.39597663040768016]]

In [39]:
dct = {"h": []}
def add():
    dct["h"].append(1)
add()
add()
add()
dct

{'h': [1, 1, 1]}

In [79]:
!NUM_PROJECTS_PER_JOB=10

In [85]:
k = 1
td = timedelta(hours=2)*${NUM_PROJECTS_PER_JOB}

SyntaxError: invalid syntax (1113283600.py, line 2)

In [77]:
hours, minutes = td.seconds//3600, (td.seconds//60)%60

In [78]:
print(f"{hours:0>2}:{minutes:0>2}:00")

02:00:00
